In [1]:
import pandas as pd
import numpy as np
from skimage import io
from collections import OrderedDict, namedtuple
from torch.optim import lr_scheduler
from glob import glob  # find all pathnames matching certain patterns
from sklearn.model_selection import GroupKFold
import joblib
import torch
import torch.nn as nn
import os
import random
import matplotlib.pyplot as plt
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2  # for converting image to tensor
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler  # return indices in sequence/random order
import cv2
from sklearn.metrics import *
from tqdm.notebook import tqdm
import transformers
from transformers import AdamW, get_linear_schedule_with_warmup, get_constant_schedule
import warnings
warnings.filterwarnings("ignore")

SEED = 42

# Seed everything for reproducable results
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [2]:
!pip install efficientnet_pytorch
# import efficientnet_pytorch
from efficientnet_pytorch import EfficientNet

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-py3-none-any.whl size=12419 sha256=95b21c0bac25a632d8694d231d873fef308f2ba83a2d6208152a219d15482d2f
  Stored in directory: /root/.cache/pip/wheels/90/6b/0c/f0ad36d00310e65390b0d4c9218ae6250ac579c92540c9097a
Successfully built efficientnet-pytorch


In [3]:
def get_net():
    net = EfficientNet.from_pretrained('efficientnet-b2')
    net._fc = nn.Linear(in_features=1408, out_features=4, bias=True)
    return net

mx = get_net()

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b2-8bb594d6.pth" to /root/.cache/torch/checkpoints/efficientnet-b2-8bb594d6.pth



Loaded pretrained weights for efficientnet-b2


In [4]:
class DatasetSubmissionRetriever(Dataset):

    def __init__(self, image_names, transforms=None):
        super().__init__()
        self.image_names = image_names
        self.transforms = transforms

    def __getitem__(self, index: int):
        image_name = self.image_names[index]
        image = cv2.imread(f'{DATA_ROOT_PATH}/Test/{image_name}', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        if self.transforms:
            sample = {'image': image}
            sample = self.transforms(**sample)
            image = sample['image']

        return image_name, image

    def __len__(self) -> int:
        return self.image_names.shape[0]
    
    def get_names(self):
        return self.image_names

In [5]:
def get_valid_transforms():
    return A.Compose([
        A.Resize(height = 512, width = 512, p = 1.0),
        ToTensorV2(p = 1.0)
    ], p = 1.0)

In [6]:
dataset = DatasetSubmissionRetriever(
    image_names=np.array([path.split('/')[-1] for path in glob('../input/alaska2-image-steganalysis/Test/*.jpg')]),
    transforms=get_valid_transforms(),
)


data_loader = DataLoader(
    dataset,
    batch_size=8,
    shuffle=False,
    num_workers=4,
    drop_last=False,
)

In [7]:
device = torch.device('cuda:0')
model = mx.to(device)

In [8]:

checkpoint1 = torch.load(f'../input/pytorch-gpu-training/model0.bin')
checkpoint2 = torch.load(f'../input/pytorch-gpu-training/model1.bin')
checkpoint3 = torch.load(f'../input/pytorch-gpu-training/model2.bin')
checkpoint4 = torch.load(f'../input/pytorch-gpu-training/model3.bin')
checkpoint5 = torch.load(f'../input/pytorch-gpu-training/model4.bin')


checkpoints = [checkpoint1, checkpoint2, checkpoint3, checkpoint4, checkpoint5]
preds = np.zeros(5000)
results = {'Id': [], 'Label': []}
i = 0
for checkpoint in checkpoints:
    result = []
    model.load_state_dict(checkpoint);
    DATA_ROOT_PATH = "../input/alaska2-image-steganalysis"
    for step, (image_names, images) in enumerate(data_loader):
        y_pred = model(images.cuda())
        y_pred = 1 - nn.functional.softmax(y_pred, dim=1).data.cpu().numpy()[:,0]
        result.extend(y_pred)
        
        if i == 0:
            results['Id'].extend(image_names)
    i += 1
    preds = preds + np.array(result)    
    print("#" * 25)
    print("\n")
        
    
        
results['Label'] = preds/5

#########################


#########################


#########################


#########################


#########################




In [9]:
submission = pd.DataFrame(results).set_index("Id")
submission.to_csv("submission.csv")